<a href="https://colab.research.google.com/github/SrijaniChakrabarti/Prediction-System-/blob/master/networkanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
import pandas as pd


In [2]:
netfile = '/content/sample_data/Chicagosketchnet.tntp'
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)


In [3]:
net

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,547,49500,0.86267,0.00,0.15,4,0,0,3
1,2,548,49500,0.86267,0.00,0.15,4,0,0,3
2,3,549,49500,0.86267,0.00,0.15,4,0,0,3
3,4,550,49500,0.86267,0.00,0.15,4,0,0,3
4,5,551,49500,0.86267,0.00,0.15,4,0,0,3
...,...,...,...,...,...,...,...,...,...,...
2945,931,906,500,22.65430,14.78,0.15,4,0,0,1
2946,932,386,49500,0.86267,0.00,0.15,4,0,0,3
2947,932,515,5000,10.44260,5.93,0.15,4,0,0,2
2948,933,387,49500,0.86267,0.00,0.15,4,0,0,3


In [4]:
net.head()

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,547,49500,0.86267,0.0,0.15,4,0,0,3
1,2,548,49500,0.86267,0.0,0.15,4,0,0,3
2,3,549,49500,0.86267,0.0,0.15,4,0,0,3
3,4,550,49500,0.86267,0.0,0.15,4,0,0,3
4,5,551,49500,0.86267,0.0,0.15,4,0,0,3


In [5]:
net.columns

Index(['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type'],
      dtype='object')

In [6]:
len(net)


2950

In [7]:
net.columns[0]

'init_node'

In [8]:
# taking all edges of graph in list
init_node_list = list(net.init_node)
term_node_list = list(net.term_node)
t = len(net)
edge_list = []
i = 0
while i < t:
  term_temp = term_node_list[i]
  init_temp = init_node_list[i]
  edge_list.append((init_temp,term_temp))
  i += 1


In [9]:
# constructing adjacency list out of it
class adjlistnode:
  def __init__(self , data):
    self.vert = data
    self.next = None

class graph:
  def __init__(self , vertices):
    self.V = vertices
    self.graph = [None]* self.V
  # add adjacency list of each edge in list
  def add_edge(self,src,dest):
    # adding the node  to the source node
    node = adjlistnode(dest)
    node.next = self.graph[src]
    self.graph[src] = node
    node = adjlistnode(src)
    node.next = self.graph[dest]
    self.graph[dest] = node
  def print_graph(self):
    for i in range(self.V):
      temp = self.graph[i]
      while temp:
        print("-> {}".format(temp.vert),end = " ")
        temp = temp.next
      print("\n")

In [10]:
gf = graph(len(net))
dest_list = []
ori_list = []
adjetailedlist = []
 
for (origin_i,dest_i) in edge_list:
  ori_list.append(origin_i)
  dest_list.append(dest_i)

for(or_i,d_i) in edge_list:
  empty_dict ={}
  x_dest = d_i
  temp = d_i
  adlist = []
  if x_dest in ori_list:
   index = ori_list.index(x_dest)
   i= index
   while (i<len(ori_list)):
      x_dest = dest_list[i]
      adlist.append(x_dest)
      i+= 1
  adlist.append(temp)
  empty_dict[or_i] = adlist
  adjetailedlist.append(empty_dict)
  



      

In [11]:
i = 0
t = len(adjetailedlist)


In [12]:
adjdict = {}
while i<t:
  temp = ori_list[i] 
  mergelist = [] 
  for key in adjetailedlist[i].keys():
    if (temp == key):
      l = adjetailedlist[i][temp]
      mergelist.append(l)
  if temp not in adjdict.keys():
    adjdict[temp] = mergelist
  i += 1
  

In [20]:
adjdict[1][0]

[1,
 548,
 549,
 621,
 2,
 547,
 550,
 552,
 618,
 3,
 547,
 550,
 551,
 4,
 548,
 549,
 551,
 552,
 553,
 560,
 5,
 495,
 549,
 550,
 563,
 6,
 435,
 548,
 550,
 553,
 618,
 619,
 7,
 496,
 550,
 552,
 560,
 8,
 435,
 437,
 614,
 619,
 622,
 625,
 9,
 556,
 622,
 624,
 625,
 10,
 437,
 496,
 555,
 557,
 560,
 624,
 11,
 490,
 556,
 558,
 559,
 624,
 630,
 12,
 491,
 557,
 560,
 561,
 13,
 491,
 557,
 562,
 566,
 631,
 14,
 495,
 550,
 553,
 556,
 558,
 561,
 15,
 492,
 494,
 558,
 560,
 16,
 492,
 493,
 559,
 563,
 567,
 17,
 494,
 551,
 562,
 564,
 18,
 493,
 563,
 565,
 19,
 564,
 568,
 569,
 20,
 500,
 559,
 567,
 631,
 21,
 499,
 562,
 566,
 22,
 533,
 565,
 574,
 23,
 499,
 532,
 533,
 565,
 570,
 572,
 573,
 24,
 500,
 569,
 571,
 572,
 25,
 501,
 570,
 572,
 631,
 632,
 634,
 637,
 26,
 569,
 570,
 571,
 573,
 576,
 637,
 27,
 528,
 531,
 569,
 572,
 577,
 28,
 531,
 532,
 568,
 575,
 29,
 528,
 530,
 574,
 580,
 581,
 30,
 572,
 577,
 578,
 637,
 643,
 31,
 530,
 545,
 573,
 5

In [19]:
new_dict = {}
for key in adjdict.keys():
  dup_removelist = []
  t = len(adjdict[key][0])
  i = 0
  while(i<t):
    x = adjdict[key][0][i]
    if x not in dup_removelist:
      dup_removelist.append(x)
    i += 1
  print(dup_removelist)
  



[1, 548, 549, 621, 2, 547, 550, 552, 618, 3, 551, 4, 553, 560, 5, 495, 563, 6, 435, 619, 7, 496, 8, 437, 614, 622, 625, 9, 556, 624, 10, 555, 557, 11, 490, 558, 559, 630, 12, 491, 561, 13, 562, 566, 631, 14, 15, 492, 494, 16, 493, 567, 17, 564, 18, 565, 19, 568, 569, 20, 500, 21, 499, 22, 533, 574, 23, 532, 570, 572, 573, 24, 571, 25, 501, 632, 634, 637, 26, 576, 27, 528, 531, 577, 28, 575, 29, 530, 580, 581, 30, 578, 643, 31, 545, 32, 507, 524, 579, 645, 33, 582, 647, 656, 34, 35, 36, 541, 660, 37, 540, 38, 394, 395, 586, 588, 605, 710, 712, 768, 808, 39, 401, 587, 771, 40, 584, 585, 772, 41, 400, 592, 604, 42, 397, 43, 590, 591, 594, 775, 44, 589, 776, 45, 441, 608, 613, 46, 609, 47, 429, 431, 595, 48, 427, 593, 596, 49, 432, 612, 617, 50, 51, 599, 778, 52, 616, 620, 53, 597, 598, 54, 601, 602, 55, 600, 603, 716, 56, 607, 671, 57, 672, 718, 58, 399, 606, 59, 60, 403, 675, 61, 676, 62, 537, 63, 611, 64, 615, 679, 65, 610, 680, 66, 440, 67, 68, 439, 554, 69, 536, 70, 433, 71, 72, 434, 